In [4]:
from pymongo import MongoClient
import pandas as pd
from collections import Counter
import seaborn as sns
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster


<h1>First we connect to MongoDB<h1>

In [5]:
client = MongoClient("localhost:27017")
client.list_database_names()


['admin', 'config', 'ironhack', 'local']

In [6]:
db = client.get_database("ironhack")
db.list_collection_names()


['companies']

In [7]:
comps = db.get_collection("companies")
comps

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'companies')

In [8]:
comps.count_documents({})

18801

<h1>Query DB<h1>

Number of employees less than 500, video game companies with more than $1M raised funds

In [9]:
comps_query = comps.find(
    {"$and":[
        {"number_of_employees": {"$gte":20}}, 
        {"number_of_employees": {"$lt": 500}},
        {"category_code": "games_video"},
        {"deadpooled_year": None},
        {
            "$or": [ 
                {"total_money_raised": {"$regex": "[A-Za-z]"}},
                {"total_money_raised" : {"$gte": 1000000}},
                    ]
        },              
            ]
    }, {"_id":0, "name":1, "number_of_employees":1, "deadpooled_year":1,  "category_code": 1, "tag_list":1, "total_money_raised": 1, "offices": 1})

comps_query.count()

<ipython-input-9-09d57ac268e3>:16: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  comps_query.count()


75

In [10]:
comps_list = list(comps_query)

<h1>Save results to a DataFrame<h1>

In [13]:
df = pd.DataFrame(comps_list)
df.sample(20)

,name,category_code,number_of_employees,deadpooled_year,tag_list,total_money_raised,offices
38,Citizen Sports,games_video,27,None,sports,$10M,"[{'description': '', 'address1': '525 Brannan ..."
18,mig33,games_video,100,None,"socialentertainment, platform, clients, androi...",$32.4M,"[{'description': 'mig33 Headquarters', 'addres..."
52,Apaja,games_video,44,None,"online, games, online-games, casual-games, gam...",€1.75M,"[{'description': 'Apaja HQ', 'address1': 'Miko..."
8,Vuze,games_video,20,None,,$34M,"[{'description': 'Headquarters', 'address1': '..."
12,MocoSpace,games_video,25,None,mobile-social-networking,$10.5M,"[{'description': '', 'address1': '', 'address2..."
1,Pando Networks,games_video,23,None,"p2p, video, streaming, download, cdn",$11M,"[{'description': None, 'address1': '520 Broadw..."
56,BASH Gaming,games_video,25,None,"casino, bingo, facebook, iphone, android, social",$1M,"[{'description': 'US Office', 'address1': '', ..."
70,Saavn,games_video,25,None,"bollywood, music, streaming, south-asian, india",$6M,"[{'description': 'Corporate Headquarters', 'ad..."
47,Atari,games_video,400,None,,$7.26M,"[{'description': 'HQ', 'address1': '417 Fifth ..."
27,Boonty,games_video,150,None,"games, free-online-games, community, social-ca...",$10M,"[{'description': None, 'address1': '54 Frankli..."


<h1>Get the cities witht the most offices<h1>

In [15]:
city_list = list()
for i, offices in enumerate(df["offices"]):
    for office in offices:
        city_list.append(office["city"])
cities_df = pd.DataFrame(city_list, columns=['City'])
cities_df["City"].value_counts()


San Francisco    14
New York         11
Los Angeles       6
Paris             5
Seattle           4
                 ..
Beirut            1
Fremont           1
Copenhagen        1
                  1
Emeryville        1
Name: City, Length: 62, dtype: int64

<h1>Make new DataFrame with office coordinates<h1>

In [17]:
data_list = list()
for index, row in df.iterrows():
    for office in row["offices"]:
        data_list.append([row["name"], row["number_of_employees"], office["city"],office["latitude"], office["longitude"]])
        
companies_df = pd.DataFrame(data_list, columns= ["name", "number_of_employees", "city", "latitude", "longitude"])      

In [18]:
companies_df

,name,number_of_employees,city,latitude,longitude
0,Kyte,40,San Francisco,37.788482,-122.409173
1,Pando Networks,23,New York,40.722655,-73.998730
2,Livestream,120,New York,40.726155,-73.995625
3,Ustream,250,San Francisco,37.392936,-122.079480
4,Ustream,250,Los Angeles,NaN,NaN
...,...,...,...,...,...
108,Blip,47,Santa Monica,34.027225,-118.468233
109,Exent,100,New York,40.752380,-74.005568
110,Exent,100,Petach-Tikva,NaN,NaN
111,Exent,100,San Francisco,37.787646,-122.402759


In [19]:
companies_df["city"].value_counts()

San Francisco    14
New York         11
Los Angeles       6
Paris             5
Seattle           4
                 ..
Beirut            1
Fremont           1
Copenhagen        1
                  1
Emeryville        1
Name: city, Length: 62, dtype: int64

<h1>Get All the offices for San Francisco<h1>

In [44]:
sanfran = companies_df.where(companies_df["city"]=="San Francisco").dropna(subset=['latitude', 'longitude'])
sanfran


,name,number_of_employees,city,latitude,longitude
0,Kyte,40.0,San Francisco,37.788482,-122.409173
3,Ustream,250.0,San Francisco,37.392936,-122.079480
8,hi5,100.0,San Francisco,37.788668,-122.400558
9,Curse,58.0,San Francisco,37.787092,-122.399972
14,Kongregate,20.0,San Francisco,37.786942,-122.401245
18,Crunchyroll,50.0,San Francisco,37.781265,-122.393229
23,Rupture,25.0,San Francisco,37.783898,-122.395234
27,Zynga,115.0,San Francisco,37.765158,-122.404234
50,Serious Business,22.0,San Francisco,37.789321,-122.401362
53,Citizen Sports,27.0,San Francisco,37.777513,-122.397044


<h1>Get All the offices in New York City<h1>

In [45]:
sanfran.to_csv("data/sanfran.csv")

In [46]:
newyork = companies_df.where(companies_df["city"]=="New York").dropna(subset=['latitude', 'longitude'])
newyork

,name,number_of_employees,city,latitude,longitude
1,Pando Networks,23.0,New York,40.722655,-73.998730
2,Livestream,120.0,New York,40.726155,-73.995625
16,OMGPOP,50.0,New York,40.723384,-74.001704
37,Boonty,150.0,New York,40.717248,-74.002662
43,Cellufun,30.0,New York,40.739930,-73.993049
82,Major League Gaming,45.0,New York,40.752672,-73.975930
105,Saavn,25.0,New York,40.743877,-73.986180
109,Exent,100.0,New York,40.752380,-74.005568


<h1>Get all the offices in Paris<h1>

In [25]:
paris = companies_df.where(companies_df["city"]=="Paris").dropna(subset=['latitude', 'longitude'])
paris

,name,number_of_employees,city,latitude,longitude
26,Kewego,65.0,Paris,46.227638,2.213749
87,Owlient,40.0,Paris,48.830809,2.303220
106,2 Minutes,105.0,Paris,48.850206,2.369803


In [42]:
sanfran_map = folium.Map(location= [37.788482,-122.409173], zoom_start= 13, height="50%", width="75%", tiles="cartodbpositron")
for index, row in sanfran.iterrows():
    Marker(location = [row["latitude"], row["longitude"]], icon=folium.Icon(color='lightred', icon='building', prefix='fa'), tooltip=row["name"]).add_to(sanfran_map)

newyork_map = folium.Map(location= [40.722655,-73.998730], zoom_start= 10, height="50%", width="75%", tiles="cartodbpositron")
for index, row in newyork.iterrows():
    Marker(location = [row["latitude"], row["longitude"]], icon=folium.Icon(color='lightblue', icon='building', prefix='fa'), tooltip=row["name"]).add_to(newyork_map)
    
paris_map = folium.Map(location= [48.830809,2.303220], zoom_start= 12, height="50%", width="75%", tiles="cartodbpositron")
for index, row in paris.iterrows():
    Marker(location = [row["latitude"], row["longitude"]], icon=folium.Icon(color='lightgreen', icon='building', prefix='fa'), tooltip=row["name"]).add_to(paris_map)


<h1>San Francisco Map<h1>

In [40]:
sanfran_map

<h1>New York City<h1>

In [41]:

newyork_map


<h1>Paris<h1>

In [43]:
paris_map